In [38]:
import pandas as pd
import numpy as np

In [64]:
X = pd.read_csv('../Heatmaps/RCS_compiled.csv')
print(len(X))
X.head()

27927


,Unnamed: 0,id,organization,last_name,first_name,email,phone,gender,ethnicity,grad_year,aptitude_fit,interest_fit,fit_rank
0,0,409512,Rocky Fork Middle School,Cardoso,Sofia,scardosoperez8225@student.rcschools.net-moved-...,615-900-7342,f,other,2023.0,Agriculture & Natural Resources,Human Services,3
1,1,409512,Rocky Fork Middle School,Cardoso,Sofia,scardosoperez8225@student.rcschools.net-moved-...,615-900-7342,f,other,2023.0,Finance,Arts & Media,2
2,2,409512,Rocky Fork Middle School,Cardoso,Sofia,scardosoperez8225@student.rcschools.net-moved-...,615-900-7342,f,other,2023.0,Engineering,Teaching,1
3,3,409514,Rocky Fork Middle School,Acevedo,Ubaldo,ubarajas5459@student.rcschools.net-moved-to-hs,615-904-6780,m,hispanic,2023.0,Agriculture & Natural Resources,Law & Public Safety,3
4,4,409514,Rocky Fork Middle School,Acevedo,Ubaldo,ubarajas5459@student.rcschools.net-moved-to-hs,615-904-6780,m,hispanic,2023.0,Architecture & Construction,Arts & Media,2


In [104]:
len(X[X.fit_rank == 1])

9309

In [101]:
test = X[X.aptitude_fit == 'Hospitality & Tourism']
# test = test[test.fit_rank == 1]
test

,Unnamed: 0,id,organization,last_name,first_name,email,phone,gender,ethnicity,grad_year,aptitude_fit,interest_fit,fit_rank
78,78,410241,Rocky Fork Middle School,Stranko,Mary,mstranko8945@student.rcschools.net-moved-to-hs,629-867-4737,f,other,2023.0,Hospitality & Tourism,Human Services,3
90,90,410301,Rocky Fork Middle School,Brown,Hailey,hbrown4051@student.rcschools.net-moved-to-hs,161-526-0408,f,white,2023.0,Hospitality & Tourism,Law & Public Safety,3
111,111,410409,Rocky Fork Middle School,smith,jiane,jsmith8477@student.rcschools.net-moved-to-hs,832-301-2877,f,black,2023.0,Hospitality & Tourism,Arts & Media,3
126,126,411996,Rocky Fork Middle School,GRIMES,MIGUEL,mgrimes6285@student.rcschools.net-moved-to-hs,615-683-8845,m,hispanic,2023.0,Hospitality & Tourism,Architecture & Construction,3
300,300,413868,Rocky Fork Middle School,zewde,nahom,nzewde666@student.rcschools.net-moved-to-hs,615-915-7708,m,black,2023.0,Hospitality & Tourism,Agriculture & Natural Resources,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...
27723,27723,1490426,Oakland Middle School,BARRETT,GRACIE,gbarrett9685@student.rcschools.net,615-557-0201,f,white,2026.0,Hospitality & Tourism,Human Services,3
27759,27759,1490465,Oakland Middle School,DELFIN VALERIO,YAJAIRA,ydelfinvalerio000@student.rcschools.net,615-546-2012,f,hispanic,2026.0,Hospitality & Tourism,Law & Public Safety,3
27798,27798,1490505,Oakland Middle School,NANCE,ELLIE,enance6859@student.rcschools.net,615-289-8455,f,white,2026.0,Hospitality & Tourism,Arts & Media,3
27879,27879,1490719,Oakland Middle School,HENDERSON,NIKILYA,nhenderson001@student.rcschools.net,615-904-6760,f,other,2026.0,Hospitality & Tourism,Architecture & Construction,3


In [69]:
def get_breakdown_by_rank(df, rank=1):
    # compiling frequency of interest/aptitude pairs
    
    fits = list(set(df.aptitude_fit).union(df.interest_fit))
    df = df[df.fit_rank == rank]
    
    breakdown_df = None
    for fit in fits:
        Y = df.loc[X.aptitude_fit == fit].interest_fit.value_counts()
        Y = pd.DataFrame(Y)
        Y.rename(columns={'interest_fit':'Apt: ' + fit},inplace=True)

        if breakdown_df is None: 
            breakdown_df = Y
        else:
            breakdown_df = pd.concat([breakdown_df,Y], axis=1)

    breakdown_df.replace(np.nan,0,inplace=True)
    
    return breakdown_df

In [90]:
r1_df

,Apt: Business,Apt: Agriculture & Natural Resources,Apt: Hospitality & Tourism,Apt: Engineering,Apt: Teaching,Apt: Finance,Apt: Distribution & Logistics,Apt: Arts & Media,Apt: Sales & Marketing,Apt: Architecture & Construction,Apt: Computers & Technology,Apt: Law & Public Safety,Apt: Health Science,Apt: Advanced Manufacturing,Apt: Government & Public Admin,Apt: Human Services
Teaching,316,29.0,0.0,245,593,49.0,186.0,11.0,408,1.0,114,0.0,1349,341.0,0.0,0.0
Sales & Marketing,116,14.0,0.0,76,240,35.0,82.0,4.0,221,2.0,55,0.0,745,148.0,0.0,0.0
Engineering,77,5.0,0.0,115,124,12.0,60.0,1.0,91,1.0,32,0.0,378,119.0,0.0,0.0
Health Science,63,4.0,0.0,56,85,15.0,34.0,3.0,65,0.0,18,0.0,279,82.0,0.0,0.0
Distribution & Logistics,61,5.0,0.0,49,119,13.0,38.0,2.0,104,1.0,29,0.0,403,104.0,1.0,0.0
Business,57,4.0,0.0,26,54,10.0,21.0,2.0,55,1.0,26,0.0,194,59.0,0.0,0.0
Advanced Manufacturing,30,1.0,0.0,34,38,3.0,21.0,1.0,35,0.0,11,0.0,141,42.0,0.0,0.0
Architecture & Construction,16,0.0,0.0,12,20,4.0,9.0,0.0,20,0.0,5,0.0,59,14.0,0.0,0.0
Computers & Technology,13,2.0,0.0,10,15,1.0,11.0,0.0,16,0.0,2,0.0,60,18.0,0.0,0.0
Agriculture & Natural Resources,5,0.0,0.0,1,5,2.0,0.0,0.0,2,0.0,1,0.0,9,4.0,0.0,0.0


In [103]:
r1_df = get_breakdown_by_rank(X)
util_dict = {}
for col in list(r1_df.columns):
    total = int(r1_df[col].sum())
    try:
        cat = r1_df[col].loc[col.split(': ')[1]]
        print(col, '- Total: ', total, '; %Interested: ', "{:.2%}".format(float(cat)/total))
    except KeyError:
        print(f'~~~At fit_rank 1, none were labeled with {col}')

Apt: Business - Total:  761 ; %Interested:  7.49%
Apt: Agriculture & Natural Resources - Total:  64 ; %Interested:  0.00%
~~~At fit_rank 1, none were labeled with Apt: Hospitality & Tourism
Apt: Engineering - Total:  629 ; %Interested:  18.28%
Apt: Teaching - Total:  1307 ; %Interested:  45.37%
Apt: Finance - Total:  145 ; %Interested:  0.69%
Apt: Distribution & Logistics - Total:  467 ; %Interested:  8.14%
Apt: Arts & Media - Total:  25 ; %Interested:  4.00%
Apt: Sales & Marketing - Total:  1027 ; %Interested:  21.52%
Apt: Architecture & Construction - Total:  6 ; %Interested:  0.00%
Apt: Computers & Technology - Total:  296 ; %Interested:  0.68%
~~~At fit_rank 1, none were labeled with Apt: Law & Public Safety
Apt: Health Science - Total:  3647 ; %Interested:  7.65%
Apt: Advanced Manufacturing - Total:  934 ; %Interested:  4.50%
~~~At fit_rank 1, none were labeled with Apt: Government & Public Admin
~~~At fit_rank 1, none were labeled with Apt: Human Services


In [105]:
# checking no students were dropped 
sum = 0
for col in list(r1_df.columns):
    sum += r1_df[col].sum()
sum

9309.0

In [110]:
# how many students across RCS had interest/aptitude align with fit_rank 1? 
perfect_fit = 0
for col in list(r1_df.columns):
    try:
        perfect_fit += int(r1_df[col].loc[col.split(': ')[1]])   
    except KeyError:
        pass

print(perfect_fit, 9309-perfect_fit, "{:.2%}".format(perfect_fit/7960))

# This does not mean that those 1349 could get into their appropriate POS placements

1349 7960 16.95%


In [48]:
# compiling frequency of interest/aptitude pairs
fits = list(set(X.aptitude_fit).union(X.interest_fit))
breakdown_df = None
for fit in fits:
    Y = X.loc[X.aptitude_fit == fit].interest_fit.value_counts()
    Y = pd.DataFrame(Y)
    Y.rename(columns={'interest_fit':'Apt: ' + fit},inplace=True)

    if breakdown_df is None: 
        breakdown_df = Y
    else:
        breakdown_df = pd.concat([breakdown_df,Y], axis=1)

breakdown_df.replace(np.nan,0,inplace=True)

breakdown_df

,Apt: Business,Apt: Agriculture & Natural Resources,Apt: Hospitality & Tourism,Apt: Engineering,Apt: Teaching,Apt: Finance,Apt: Distribution & Logistics,Apt: Arts & Media,Apt: Sales & Marketing,Apt: Architecture & Construction,Apt: Computers & Technology,Apt: Law & Public Safety,Apt: Health Science,Apt: Advanced Manufacturing,Apt: Government & Public Admin,Apt: Human Services
Teaching,316.0,29,0.0,245,593.0,49,186.0,11,408.0,1,114,0.0,1349,341,0.0,0.0
Health Science,156.0,161,0.0,195,85.0,234,63.0,43,101.0,61,244,0.0,352,339,16.0,19.0
Arts & Media,149.0,575,48.0,204,9.0,388,48.0,201,69.0,177,527,175.0,132,399,134.0,146.0
Sales & Marketing,143.0,61,0.0,106,240.0,86,89.0,20,238.0,17,128,0.0,755,218,6.0,5.0
Engineering,130.0,89,0.0,210,124.0,152,75.0,20,105.0,38,150,0.0,412,272,14.0,8.0
Business,97.0,141,3.0,78,54.0,195,40.0,39,73.0,56,141,8.0,243,272,34.0,13.0
Human Services,69.0,816,69.0,97,0.0,300,25.0,297,27.0,211,433,313.0,48,266,215.0,226.0
Distribution & Logistics,69.0,59,0.0,63,119.0,73,42.0,13,116.0,29,74,1.0,415,176,4.0,5.0
Agriculture & Natural Resources,44.0,314,22.0,84,5.0,195,15.0,67,11.0,120,180,79.0,40,180,90.0,47.0
Architecture & Construction,43.0,147,16.0,71,20.0,139,20.0,27,34.0,75,141,22.0,85,142,57.0,17.0


In [63]:
int_total, apt_total = {}, {}

for fit in fits:
    int_total[fit] = breakdown_df.loc['Business'].sum()
    apt_total['Apt: ' + fit] = breakdown_df['Apt: ' + fit].sum()
    
    print(int_total[fit], apt_total['Apt: ' + fit], fit)

1487.0 1334.0 Business
1487.0 3840 Agriculture & Natural Resources
1487.0 330.0 Hospitality & Tourism
1487.0 1558 Engineering
1487.0 1307.0 Teaching
1487.0 2395 Finance
1487.0 675.0 Distribution & Logistics
1487.0 1090 Arts & Media
1487.0 1254.0 Sales & Marketing
1487.0 1134 Architecture & Construction
1487.0 2776 Computers & Technology
1487.0 1137.0 Law & Public Safety
1487.0 4120 Health Science
1487.0 3118 Advanced Manufacturing
1487.0 1051.0 Government & Public Admin
1487.0 808.0 Human Services
